In [ ]:
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import model_selection.cross_validate
from sklearn.metrics import precision_recall_fscore_support,accuracy_score

In [ ]:


#Create Tree
tree = XGBClassifier(n_estimators=5, max_depth=2, learning_rate=1, objective='multi')

#Train model
tree.fit(train["text"],train["label"])

pred_XGBoost = tree.predict(train["label"])

In [ ]:
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
cross_val_results = cross_val_score(tree, X, y, cv=kf)